In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [36]:
# %%time
# questions = pd.read_csv('./question_train_word.csv')

# train_question_topics = pd.read_csv('./question_topic_train_set.txt', sep='\t', names=['q_id', 'topics'])

# def split_topics(topics):
#     topics = topics.split(',')
#     result = []
#     for t in topics:
#         if t not in result:
#             result.append(t)
#         if len(result) >= 5:
#             break
#     return ','.join(result)
# questions['topics'] = train_question_topics.topics.apply(split_topics)
# del train_question_topics

# questions.to_csv('./question_train_word.csv')


CPU times: user 51.8 s, sys: 6.39 s, total: 58.2 s
Wall time: 1min 2s


In [3]:
%%time
questions = pd.read_csv('./question_train_word.csv')
questions['topics'] = questions.topics.apply(lambda s: s.split(','))

CPU times: user 21.1 s, sys: 4.76 s, total: 25.8 s
Wall time: 27.7 s


In [40]:
questions.iloc[0]

Unnamed: 0                                                     0
id                                           6555699376639805223
title          w305,w13549,w22752,w11,w7225,w2565,w1106,w16,w...
description    w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1...
topics                [7739004195693774975, 3738968195649774859]
Name: 0, dtype: object

## Using Mean Word2Vec To Simplify The Input

In [5]:
# %%time

# from gensim.models import KeyedVectors
# emb_words = KeyedVectors.load_word2vec_format('./word_embedding.txt')

CPU times: user 1min 38s, sys: 2.96 s, total: 1min 41s
Wall time: 1min 45s


In [6]:
# from tqdm import tqdm

# index2word_set = set(emb_words.index2word)

# def generate_input_vector(titles, num_features):
#     result = []
#     for title in tqdm(titles):
#         featureVec = np.zeros((num_features,), dtype='float32')
#         nwords = 0
#         if type(title) != str:
#             result.append(featureVec)
#             continue
#         for word in title.split(','):
#             if (word not in index2word_set) or (not word_weight.get(word)):
#                 continue
#             featureVec = np.add(featureVec, np.multiply(emb_words[word], word_weight[word]))
#             nwords += 1
#         if nwords:
#             featureVec = np.divide(featureVec, nwords)
#         result.append(featureVec)
#     return result

In [7]:
# from collections import Counter
# words = [w for title in questions.title if type(title) == str for w in title.split(',')]
# count_words = Counter(words)
# total_words = len(words)
# word_weight = {l: np.log(total_words / v) for (l, v) in count_words.items()}

In [8]:
# %%time
# X_train = generate_input_vector(questions.title, 256)

100%|██████████| 2999967/2999967 [05:55<00:00, 8436.26it/s] 

CPU times: user 3min 48s, sys: 26.8 s, total: 4min 15s
Wall time: 5min 55s


In [13]:
# print(X_train[0].shape)
# print(sys.getsizeof(emb_words))
# print(sys.getsizeof(X_train))

(256,)
56
25105992


In [16]:
# %%time
# import csv

# with open("weight_vector.csv", "w") as f:
#     writer = csv.writer(f, quoting=csv.QUOTE_ALL)
#     writer.writerows(X_train)

CPU times: user 11min 53s, sys: 19.5 s, total: 12min 12s
Wall time: 12min 30s


In [17]:
# del emb_words

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### transform y in matrics

In [5]:
%%time
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer()
y_train = mb.fit_transform(questions.topics)
y_train.shape

CPU times: user 9.01 s, sys: 10.5 s, total: 19.5 s
Wall time: 22.7 s


In [22]:
# %%time
# clf = OneVsRestClassifier(SVC(kernel='linear', probability=True, class_weight='auto'), n_jobs=-1)
# clf.fit(X_train,y_train)

Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.

KeyboardInterrupt: 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [7]:
%%time
X_train_count = count_vect.fit_transform(questions.title.astype('U'))

CPU times: user 52.4 s, sys: 2.44 s, total: 54.9 s
Wall time: 57 s


In [8]:
X_train_count.shape

(2999967, 324960)

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

In [10]:
%%time
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)

CPU times: user 3.75 s, sys: 771 ms, total: 4.52 s
Wall time: 4.66 s


In [11]:
X_train_tfidf.shape

(2999967, 324960)

In [12]:
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier

n_estimators = 10

In [13]:
%%time
rf = RandomForestClassifier(n_estimators=n_estimators, verbose=2, n_jobs=-1)
rf.fit(X_train_tfidf[:100], y_train[:100])

building tree 2 of 10building tree 3 of 10building tree 1 of 10

building tree 4 of 10

building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
CPU times: user 1.76 s, sys: 576 ms, total: 2.34 s
Wall time: 1.24 s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.1s finished


In [49]:
sgd = OneVsRestClassifier(SGDClassifier(loss="hinge", penalty="l2", verbose=2, n_jobs=-1))

In [48]:
%%time
sgd.fit(X_train_tfidf[:100], y_train[:100])

ValueError: bad input shape (10, 1999)

In [23]:
topics = [t for ts in questions.topics for t in ts]
len(topics)

164195610

In [24]:
len(set(topics))

16

In [37]:

questions.topics[0]

'7739004195693774975,3738968195649774859'

In [28]:
topics[0]

'['